In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import sqlite3
from sqlite3 import Error
from ydata_profiling import ProfileReport
import mlflow
from mlflow.models import infer_signature

%matplotlib inline
# Load dataset
beforeDataClean = pd.read_csv("D:\\Code\\ESDS_503\\GIT\\diseasePredictionSystem\\diabetes.csv")
# Generate Profile Report
print("Generating profile report...")
profile = ProfileReport(beforeDataClean, title="Diabetes Dataset Profiling Report")
profile.to_notebook_iframe()
# Compute the correlation matrix for numerical columns
numerical_columns = beforeDataClean.select_dtypes(include=['number'])
correlation_matrix = numerical_columns.corr()
plt.figure(figsize=(10, 8))  # Set the figure size
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix Heatmap")
plt.show()
# Data Cleaning
afterDataClean = beforeDataClean.copy(deep=True)
afterDataClean[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = (
    afterDataClean[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN))
afterDataClean['Glucose'] = afterDataClean['Glucose'].fillna(round(afterDataClean['Glucose'].mean()))
afterDataClean['BloodPressure'] = afterDataClean['BloodPressure'].fillna(round(afterDataClean['BloodPressure'].mean()))
afterDataClean['SkinThickness'] = afterDataClean['SkinThickness'].fillna(round(afterDataClean['SkinThickness'].median()))
afterDataClean['Insulin'] = afterDataClean['Insulin'].fillna(round(afterDataClean['Insulin'].median()))
afterDataClean['BMI'] = afterDataClean['BMI'].fillna(round(afterDataClean['BMI'].median()))

# Check for remaining NaN values
print("Remaining NaN values:", afterDataClean.isnull().sum())


# Final data for processing
data = afterDataClean

Generating profile report...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Database Connection Functions
def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)
    return conn

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

# Create database and tables
dbConnection = create_connection("normalizedData.db", True)

patientTable = '''CREATE TABLE Patient (
    PatientID INTEGER PRIMARY KEY AUTOINCREMENT,
    Pregnancies INTEGER,
    Age INT
);'''
healthMatrixTable = '''CREATE TABLE HealthMetrics (
    PatientID INTEGER,
    Glucose INTEGER,
    BloodPressure INTEGER,
    SkinThickness INTEGER,
    Insulin INTEGER,
    BMI FLOAT,
    DiabetesPedigreeFunction FLOAT,
    FOREIGN KEY (PatientID) REFERENCES Patient(PatientID)
);'''
diseaseOutcomeTable = '''CREATE TABLE Outcome (
    PatientID INTEGER,
    DiabetesStatus INTEGER,
    FOREIGN KEY (PatientID) REFERENCES Patient(PatientID)
);'''

create_table(dbConnection, patientTable)
create_table(dbConnection, healthMatrixTable)
create_table(dbConnection, diseaseOutcomeTable)

# Insert data into tables
def insert_patient_data(conn, data):
    cursor = conn.cursor()
    for _, row in data.iterrows():
        cursor.execute(
            "INSERT INTO Patient (Pregnancies, Age) VALUES (?, ?)",
            (row["Pregnancies"], row["Age"])
        )
    conn.commit()

def insert_health_metrics(conn, data):
    cursor = conn.cursor()
    patient_ids = cursor.execute("SELECT PatientID FROM Patient").fetchall()
    for _, (row, patient_id) in enumerate(zip(data.iterrows(), patient_ids)):
        row = row[1]
        cursor.execute(
            """
            INSERT INTO HealthMetrics (
                PatientID, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            (
                patient_id[0],
                row["Glucose"],
                row["BloodPressure"],
                row["SkinThickness"],
                row["Insulin"],
                row["BMI"],
                row["DiabetesPedigreeFunction"]
            )
        )
    conn.commit()

def insert_outcome(conn, data):
    cursor = conn.cursor()
    patient_ids = cursor.execute("SELECT PatientID FROM Patient").fetchall()
    for _, (row, patient_id) in enumerate(zip(data.iterrows(), patient_ids)):
        row = row[1]
        cursor.execute(
            "INSERT INTO Outcome (PatientID, DiabetesStatus) VALUES (?, ?)",
            (patient_id[0], row["Outcome"])
        )
    conn.commit()

insert_patient_data(dbConnection, data)
insert_health_metrics(dbConnection, data)
insert_outcome(dbConnection, data)
print("Data inserted successfully!")

# Load data from database
dataFromTable = '''SELECT 
    p.PatientID, 
    p.Pregnancies, 
    p.Age, 
    hm.Glucose, 
    hm.BloodPressure, 
    hm.SkinThickness, 
    hm.Insulin, 
    hm.BMI, 
    hm.DiabetesPedigreeFunction, 
    o.DiabetesStatus AS Outcome
FROM Patient p
JOIN HealthMetrics hm ON p.PatientID = hm.PatientID
JOIN Outcome o ON p.PatientID = o.PatientID;'''
data = pd.read_sql_query(dataFromTable, dbConnection)
print(data.head())
dbConnection.close()
print(data.describe())

Data inserted successfully!
   PatientID  Pregnancies  Age  Glucose  BloodPressure  SkinThickness  \
0          1            6   50      148             72             35   
1          2            1   31       85             66             29   
2          3            8   32      183             64             29   
3          4            1   21       89             66             23   
4          5            0   33      137             40             35   

   Insulin   BMI  DiabetesPedigreeFunction  Outcome  
0      125  33.6                     0.627        1  
1      125  26.6                     0.351        0  
2      125  23.3                     0.672        1  
3       94  28.1                     0.167        0  
4      168  43.1                     2.288        1  
        PatientID  Pregnancies         Age     Glucose  BloodPressure  \
count  768.000000   768.000000  768.000000  768.000000     768.000000   
mean   384.500000     3.845052   33.240885  121.688802      72.

In [ ]:
# Data Preprocessing -- Feature Scaling 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(data)


std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(data)

In [ ]:
# Splitting features and target
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Define numerical and categorical features
numerical_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
categorical_features = ['Pregnancies']

# Define transformations for numerical data
numerical_transformer = Pipeline(steps=[
    ('log', FunctionTransformer(np.log1p, validate=True)),  # Log Transformation
    ('scaler', StandardScaler())  # Standard Scaling
])

# Define transformations for categorical data with OneHotEncoder's sparse_output set to False
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),  # One-Hot Encoding with dense output
    ('minmax', MinMaxScaler())  # MinMax Scaling
])

# Combine preprocessors using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# Create preprocessing pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Apply preprocessing and SMOTE
X_transformed = pipeline.fit_transform(X)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_transformed, y)


# Class distribution profiling after SMOTE
print("Class distribution after SMOTE:")
print(y_res.value_counts())

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

NameError: name 'Pipeline' is not defined

In [ ]:
# Define models
log_reg = LogisticRegression(max_iter=200, random_state=42)
svm = SVC(probability=True, random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)

# Random Forest Hyperparameter Tuning
param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_

# Stacking
stacking = StackingClassifier(
    estimators=[('lr', log_reg), ('rf', best_rf), ('svm', svm)],
    final_estimator=LogisticRegression(max_iter=200, random_state=42)
)

# Evaluate
models = {'Logistic Regression': log_reg, 'Random Forest': best_rf, 'SVM': svm, 'Gradient Boosting': gradient_boosting, 'Stacking': stacking}
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'Confusion Matrix': confusion_matrix(y_test, y_pred),
        'Classification Report': classification_report(y_test, y_pred)
    }

results_df = pd.DataFrame(results).T
print(results_df)

                    Accuracy  F1 Score      Confusion Matrix  \
Logistic Regression    0.735  0.738916  [[72, 27], [26, 75]]   
Random Forest           0.81  0.819048  [[76, 23], [15, 86]]   
SVM                     0.76  0.773585  [[70, 29], [19, 82]]   
Gradient Boosting       0.78  0.786408  [[75, 24], [20, 81]]   
Stacking               0.805  0.815166  [[75, 24], [15, 86]]   

                                                 Classification Report  
Logistic Regression                precision    recall  f1-score   ...  
Random Forest                      precision    recall  f1-score   ...  
SVM                                precision    recall  f1-score   ...  
Gradient Boosting                  precision    recall  f1-score   ...  
Stacking                           precision    recall  f1-score   ...  


In [ ]:
# Save best model
best_model_name = results_df['Accuracy'].idxmax()
joblib.dump(models[best_model_name], 'disease_prediction_model.pkl')
print(f"Best model saved: {best_model_name}")

Best model saved: Random Forest


In [ ]:
import os

MLFLOW_TRACKING_URI="https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']='s3akash'
os.environ['MLFLOW_TRACKING_PASSWORD']='4d779774d517aa973e2858c5d93af0e87a4ca457'


import mlflow
from mlflow.models import infer_signature

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)

# Create a new MLflow Experiment
mlflow.set_experiment("diseasePredictionSystemModel")

# Start an MLflow run
for algo, model in models.items():
    with mlflow.start_run():
        if isinstance(model, GridSearchCV):
            score = model.best_score_
            params = model.best_params_
            best_model = model.best_estimator_
        else:
            score = accuracy_score(y_test, model.predict(X_test))
            params = {}
            best_model = model
        
        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", score)
        mlflow.log_metric("f1_score", f1_score(y_test, model.predict(X_test)))
        
        # Infer the model signature
        signature = infer_signature(X_train, best_model.predict(X_train))
        
        # Log the model
        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path="model",
            signature=signature,
            input_example=X_train,
            registered_model_name=algo,
        )

Registered model 'Logistic Regression' already exists. Creating a new version of this model...
2024/12/18 21:02:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 10
Created version '10' of model 'Logistic Regression'.


🏃 View run amusing-hen-501 at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1/runs/fd549fef2fa74245a5ab6b20d51be5fa
🧪 View experiment at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1


Registered model 'Random Forest' already exists. Creating a new version of this model...
2024/12/18 21:03:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest, version 10
Created version '10' of model 'Random Forest'.


🏃 View run polite-stag-555 at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1/runs/0fe9cdc09a594ee0941eea5b61e141a8
🧪 View experiment at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1


Registered model 'SVM' already exists. Creating a new version of this model...
2024/12/18 21:03:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SVM, version 10
Created version '10' of model 'SVM'.


🏃 View run rumbling-panda-629 at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1/runs/1ac6c4b0258c4f23abdee5d62a2a75d0
🧪 View experiment at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1


Registered model 'Gradient Boosting' already exists. Creating a new version of this model...
2024/12/18 21:03:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Gradient Boosting, version 10
Created version '10' of model 'Gradient Boosting'.


🏃 View run bald-quail-729 at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1/runs/01fbb6719e5e4505b381c8c85e6ab3d2
🧪 View experiment at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1


Registered model 'Stacking' already exists. Creating a new version of this model...
2024/12/18 21:04:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Stacking, version 10
Created version '10' of model 'Stacking'.


🏃 View run polite-pig-474 at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1/runs/6de7d058469a4223b936f93a78ec2464
🧪 View experiment at: https://dagshub.com/s3akash/diseasePredictionSystemModel.mlflow/#/experiments/1
